In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import re
import csv
import json
import nltk
from rake_nltk import Rake
from pandas.io.json import json_normalize
from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [2]:
filename = '/home/claire/Documents/jobs/milieu/milieu-noumea/noumea-analysis/python-noumea-sept5.csv'
df = pd.DataFrame.from_csv(filename, header=0, sep=',', index_col=0)
pd.options.display.max_columns = 999
df

,participant_id,email,age,enable_receive_emails,lives_in_noumea,months_lived_in_noumea,neighbourhood,text_q1-1,text_q1-10,text_q1-11,text_q1-2,text_q1-3,text_q1-4,text_q1-5,text_q1-6,text_q1-7,text_q1-8,text_q1-9,text_q1-comment,text_q2-1,text_q2-10,text_q2-11,text_q2-12,text_q2-13,text_q2-14,text_q2-2,text_q2-3,text_q2-4,text_q2-5,text_q2-6,text_q2-7,text_q2-8,text_q2-9,text_q2-comment,text_q3-1,text_q3-10,text_q3-2,text_q3-3,text_q3-4,text_q3-5,text_q3-6,text_q3-7,text_q3-8,text_q3-9,text_q3-comment,text_q4-1,text_q4-10,text_q4-11,text_q4-12,text_q4-13,text_q4-2,text_q4-3,text_q4-4,text_q4-5,text_q4-6,text_q4-7,text_q4-8,text_q4-9,text_q4-comment,text_q5-1,text_q5-2,text_q5-3,text_q5-4,text_q5-5,text_q5-6,text_q5-7,text_q5-8,text_q5-9,text_q5-comment,watson_keywords_q1-comment,watson_keywords_q2-comment,watson_keywords_q3-comment,watson_keywords_q4-comment,watson_keywords_q5-comment,watson_score_q1-comment,watson_score_q2-comment,watson_score_q3-comment,watson_score_q4-comment,watson_score_q5-comment
3,29,jonathan.rebouillon@gmail.com,26.0,True,True,2 ans,NaN,4,3,4,4,4,2,4,3,4,1,4,NaN,3,3,2,3,4,2,4,4,2,4,4,3,4,4,NaN,2,4,2,2,2,2,4,4,4,4,NaN,4,4,4,2,4,2,4,4,3,4,2,4,3,NaN,4,0,2,1,4,4,3,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30,Olivia.kurt@gmail.com,30.0,False,True,30,NaN,2,2,2,2,2,2,2,2,2,2,2,NaN,2,2,2,2,2,2,2,2,2,2,2,2,2,2,NaN,2,2,2,2,2,2,2,2,2,2,NaN,2,2,2,2,2,2,2,4,4,4,4,4,4,NaN,2,2,2,2,3,3,3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,32,ornc44@gmail.com,32.0,False,True,22,NaN,0,0,0,0,1,0,0,1,0,0,4,NaN,2,3,4,0,0,2,2,2,4,3,0,4,0,1,NaN,0,0,0,0,0,0,4,3,3,2,NaN,2,2,2,0,4,1,1,0,3,4,3,3,4,NaN,2,0,0,0,0,0,0,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,33,e.launay@live.fr,28.0,True,False,NaN,NaN,2,2,3,3,2,2,2,2,4,2,4,il faudra mettre en place un service de sécuri...,2,4,2,1,3,3,4,4,4,4,4,4,4,4,NaN,1,2,3,0,0,2,4,4,4,4,j'espère que les gares des bus ne ressembleron...,3,4,4,4,2,4,4,4,4,4,4,2,4,NaN,2,2,3,2,2,2,0,4,4,NaN,[],NaN,[],NaN,NaN,0.0,NaN,0.0,NaN,NaN
8,34,haguet.megane@gmail.com,26.0,True,True,26 ans,Baie des citrons,3,2,3,2,3,3,3,3,4,2,4,NaN,3,3,2,2,3,3,4,3,3,3,4,4,2,4,NaN,3,3,3,2,2,2,4,4,4,4,NaN,4,3,3,3,3,2,2,2,3,3,3,2,3,NaN,2,2,3,1,2,2,2,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,35,NaN,29.0,False,True,4,Magenta,2,2,2,2,1,2,2,3,2,2,2,NaN,2,2,2,2,2,2,2,2,2,2,2,2,2,2,NaN,2,3,2,2,2,1,3,2,3,3,NaN,2,3,3,3,4,2,2,3,3,2,3,3,2,NaN,0,2,2,2,3,2,2,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,36,fgambarotto@yahoo.fr,46.0,False,True,10,Tuband,2,2,2,2,2,2,3,3,1,1,1,NaN,1,3,3,3,3,2,3,3,3,3,4,4,4,3,NaN,3,2,2,2,2,2,1,2,2,2,NaN,3,1,1,2,3,3,3,3,2,2,2,2,2,NaN,3,2,2,3,3,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,38,marie.brenon@lagoon.nc,37.0,True,True,3 ans,Trianon,4,4,3,1,1,3,3,1,3,1,4,NaN,2,2,4,0,2,2,3,4,4,4,3,4,4,4,NaN,2,2,2,2,2,2,4,4,4,2,NaN,4,1,3,3,4,3,4,3,2,2,2,2,2,NaN,4,3,3,3,2,2,4,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,40,marie.c.22@hotmail.fr,29.0,True,True,1,NaN,2,2,2,3,3,3,2,2,3,2,3,NaN,3,3,3,3,3,3,3,3,3,0,2,4,2,3,NaN,2,3,3,3,2,2,4,4,2,4,NaN,4,4,3,4,3,4,4,2,4,4,2,2,4,NaN,3,2,2,2,2,2,3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,41,micalonc@gmail.com,63.0,True,True,12,NaN,2,2,2,2,2,2,2,2,2,2,2,NaN,2,2,2,2,2,2,2,2,2,2,2,2,2,2,NaN,2,2,2,2,2,2,4,2,2,4,NaN,3,2,3,3,2,2,2,3,3,4,2,2,2,NaN,2,2,2,2,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
dfc = df[['text_q1-comment', 'text_q2-comment', 'text_q3-comment', 'text_q4-comment', 'text_q5-comment']]
q1 = pd.DataFrame(df['text_q1-comment'].to_frame().replace(np.nan, '', regex=True))
q2 = pd.DataFrame(columns=['score1', 'keyword1', 'score2', 'keyword2'])
q3 = pd.concat([q1, q2])
pd.options.display.max_rows = 999

for index, row in q3.iterrows():
    comment = row['text_q1-comment']
    r = Rake(language='french')
    if comment != "":
        r.extract_keywords_from_text(comment)
        x = r.get_ranked_phrases_with_scores()
#         print("COMMENT:", comment, "\nKEYWORD:", x[0][1], "\nSCORE:", x[0][0])
        row['keyword1'] = x[0][1]
        row['keyword2'] = x[1][1]
        row['score1'] = x[0][0]
        row['score2'] = x[1][0]
    else:
        row['keyword1'] = np.NaN
        row['keyword2'] = np.NaN
        row['score1'] = np.NaN
        row['score2'] = np.NaN
q3.dropna()

,keyword1,keyword2,score1,score2,text_q1-comment
7,faudra mettre,sécurité,4,1,il faudra mettre en place un service de sécuri...
71,public si aucune exposition,cette zone devrait regrouper,16,14.6667,Manque de visibilité pour les commerces envisa...
131,utiliser cet endroit,les futurs personnes,9,9,Dommage que des parkings disparaissent pour ce...
144,nouveau souffle,faudrait prévoir,4,4,Il faudrait prévoir un espace innovant pour do...
169,accueillir les touristes étrangers,ville les weekends,15.5,9,Et pourquoi pas un complexe de restauration-hô...
186,locaux partagés pourrait être revue,position,25,1,La position des bureaux et des locaux partagés...
263,aménager les quais citées,passerelle piéton,16,4,Pourquoi ne pas aménager les quais citées en f...
264,visualiser juste,plan,4,1,C'est difficile de visualiser juste avec le plan
281,ds logements types logements,proposer différents services,16,8.5,Y aur-t-il ds logements types logements de con...
325,les enfants,jeu,4,1,Y aura t-il des espaces de jeu pour les enfants?


In [70]:
result = q3.dropna().sort_values(['score1'], ascending=[False])
result

,keyword1,keyword2,score1,score2,text_q1-comment
186,locaux partagés pourrait être revue,position,25,1,La position des bureaux et des locaux partagés...
372,différentes zones les espaces publics,les différents usages prévus,20.8333,16,Le café devrait donner plutôt sur les espaces ...
71,public si aucune exposition,cette zone devrait regrouper,16,14.6667,Manque de visibilité pour les commerces envisa...
263,aménager les quais citées,passerelle piéton,16,4,Pourquoi ne pas aménager les quais citées en f...
281,ds logements types logements,proposer différents services,16,8.5,Y aur-t-il ds logements types logements de con...
169,accueillir les touristes étrangers,ville les weekends,15.5,9,Et pourquoi pas un complexe de restauration-hô...
131,utiliser cet endroit,les futurs personnes,9,9,Dommage que des parkings disparaissent pour ce...
7,faudra mettre,sécurité,4,1,il faudra mettre en place un service de sécuri...
144,nouveau souffle,faudrait prévoir,4,4,Il faudrait prévoir un espace innovant pour do...
264,visualiser juste,plan,4,1,C'est difficile de visualiser juste avec le plan


In [74]:
result.to_csv('nltk_q1.csv')

In [ ]:
# dfc2 = pd.DataFrame(dfc.mean()*100, columns=['mean'])
# dfc.describe()